## Игра в слова

Для начала загрузим список всех существительных в единственном числе.

In [1]:
f = open('russian_nouns.txt', 'r', encoding='utf-8')
words = []
while True:
    line = f.readline()
    if not line:
        break
    words.append(line.strip())
f.close()

In [2]:
len(words)

51301

Исключим слова, которые точно не могут быть получены в игре.

Например, "абажурчик" не получится, так как до этого уже составится "абажур".

In [3]:
for j in range(len(words)):
    rep_ind = False
    for i in range(1, len(words[j])):
        if words[j][:i] in words:
            rep_ind = True
    if rep_ind:
        words[j] = 'del'

In [4]:
words = [words[i] for i in range(len(words)) if words[i] != 'del']

In [5]:
len(words)

22597

Теперь генерируем все возможные ситуации, которые могут получиться в игре.

In [6]:
positions = ['']
for word in words:
    for i in range(1, len(word)+1):
        positions.append(word[:i])

In [7]:
positions = tuple(sorted(set(positions)))

In [8]:
import numpy as np
N = len(positions)

In [9]:
N

97892

В список res будем записывать результат игры, если игра дошла до i-ой позиции.

В список first_move будет записывать, делает ли в i-ой позиции ход первый игрок.

В список is_def будем записывать, определен ли результат игры.

In [10]:
res = np.zeros(N)
first_move = []
for i in range(N):
    if len(positions[i]) % 2 == 0:
        first_move.append(True)
    else:
        first_move.append(False)
is_def = np.zeros(N)

In [11]:
len(first_move)

97892

Теперь применим алгоритм Цермело.

Для начала определим, кто выигрывает в позициях, где игра заканчивается.

In [12]:
for i in range(N):
    if positions[i] in words:
        if first_move[i]:
            res[i] = 2
        else:
            res[i] = 1
        is_def[i] = 1

In [13]:
is_def.sum()

22597.0

In [14]:
res.sum()

33860.0

А теперь создадим список списков next_pos.

Для каждой i-ой позиции укажем список номеров позиций, в которые можно попасть в один ход из i-ой позиции.

In [15]:
next_pos = []

In [16]:
for i in range(N):
    next_i = []
    for j in range(i+1, N):
        if len(positions[i]) >  len(positions[j]):
            break
        if len(positions[i]) + 1 == len(positions[j]):
            if positions[i] == positions[j][:len(positions[j]) - 1]:
                next_i.append(j)
    next_pos.append(next_i)

In [17]:
len(next_pos)

97892

In [18]:
for m in next_pos[1]:
    print(positions[m])

аб
ав
аг
ад
аж
аз
аи
ай
ак
ал
ам
ан
ао
ап
ар
ас
ат
ау
аф
ах
ац
аш
аэ


Осталось только реализовать алгоритм Цермело.

In [19]:
while is_def.sum() < N:
    for i in range(N):
        if is_def[i] == 0:
            is_win = False
            is_lose = True
            if first_move[i]:
                for m in next_pos[i]:
                    if res[m] == 0:
                        is_lose = False
                    if res[m] == 1:
                        is_lose = False
                        is_win = True
                if is_win:
                    res[i] = 1
                    is_def[i] = 1
                if is_lose:
                    res[i] = 2
                    is_def[i] = 1
            else:
                for m in next_pos[i]:
                    if res[m] == 0:
                        is_lose = False
                    if res[m] == 2:
                        is_lose = False
                        is_win = True
                if is_win:
                    res[i] = 2
                    is_def[i] = 1
                if is_lose:
                    res[i] = 1
                    is_def[i] = 1
    print(is_def.sum())

44332.0
58723.0
71137.0
79475.0
86153.0
90591.0
93778.0
95570.0
96684.0
97247.0
97576.0
97748.0
97829.0
97866.0
97881.0
97887.0
97890.0
97892.0


In [20]:
res

array([1., 2., 1., ..., 2., 2., 1.])

In [21]:
for m in next_pos[0]:
    print(str(positions[m]) + ' ' + str(int(res[m])))

а 2
б 2
в 2
г 2
д 2
е 2
ж 2
з 2
и 2
й 1
к 2
л 2
м 2
н 2
о 2
п 2
р 2
с 2
т 2
у 2
ф 2
х 2
ц 2
ч 2
ш 2
щ 2
э 2
ю 2
я 2
ё 2


In [22]:
for m in next_pos[0]:
    print(str(positions[m]) + ' ' + str(int(res[m])) + '   позиция ' + str(int(m)))

а 2   позиция 1
б 2   позиция 2522
в 2   позиция 6253
г 2   позиция 14379
д 2   позиция 16988
е 2   позиция 20694
ж 2   позиция 21166
з 2   позиция 22178
и 2   позиция 23297
й 1   позиция 26889
к 2   позиция 26907
л 2   позиция 32030
м 2   позиция 34168
н 2   позиция 36897
о 2   позиция 48248
п 2   позиция 58082
р 2   позиция 70339
с 2   позиция 73575
т 2   позиция 83343
у 2   позиция 86289
ф 2   позиция 88516
х 2   позиция 89608
ц 2   позиция 90534
ч 2   позиция 91301
ш 2   позиция 92998
щ 2   позиция 94771
э 2   позиция 95048
ю 2   позиция 97334
я 2   позиция 97475
ё 2   позиция 97855


In [23]:
for m in next_pos[26889]:
    print(str(positions[m]) + ' ' + str(int(res[m])) + '   позиция ' + str(int(m)))

йе 1   позиция 26890
йо 1   позиция 26898


In [24]:
for m in next_pos[26890]:
    print(str(positions[m]) + ' ' + str(int(res[m])) + '   позиция ' + str(int(m)))

йем 1   позиция 26891


In [25]:
positions[26895], positions[26897]

('йеменец', 'йеменка')

In [26]:
for m in next_pos[26898]:
    print(str(positions[m]) + ' ' + str(int(res[m])) + '   позиция ' + str(int(m)))

йог 1   позиция 26899
йод 1   позиция 26900
йор 1   позиция 26901
йот 1   позиция 26906


In [27]:
next_pos

[[1,
  2522,
  6253,
  14379,
  16988,
  20694,
  21166,
  22178,
  23297,
  26889,
  26907,
  32030,
  34168,
  36897,
  48248,
  58082,
  70339,
  73575,
  83343,
  86289,
  88516,
  89608,
  90534,
  91301,
  92998,
  94771,
  95048,
  97334,
  97475,
  97855],
 [2,
  175,
  299,
  554,
  555,
  570,
  571,
  572,
  602,
  917,
  1277,
  1541,
  1968,
  1975,
  2219,
  2220,
  2221,
  2323,
  2324,
  2408,
  2409,
  2431,
  2434],
 [3, 9, 21, 24, 31, 34, 59, 88, 111, 162, 166, 169, 172],
 [4, 7, 8],
 [5],
 [6],
 [],
 [],
 [],
 [10, 12],
 [11],
 [],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [],
 [22],
 [23],
 [],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [],
 [32],
 [33],
 [],
 [35, 41, 52],
 [36],
 [37],
 [38],
 [39],
 [40],
 [],
 [42],
 [43, 46],
 [44],
 [45],
 [],
 [47],
 [48, 50],
 [49],
 [],
 [51],
 [],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [],
 [60, 71, 79],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67, 69],
 [68],
 [],
 [70],
 [],
 [72],
 [73, 77],
 [74]